# Item Collaborative Filtering on Residuals
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two items
* This alpha is trained on the residuals of previous ItemCF alphas

In [1]:
name = "ItemCF.Resid";

In [2]:
using NBInclude
@nbinclude("ItemCFBase.ipynb");

## Compute cosine correlations

In [3]:
function get_correlation_matrix_outdir(residual_alphas)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(residual_alphas))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $residual_alphas"
    training = RatingsDataset64(get_residuals("training", residual_alphas))
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    S = zeros(maximum(training.item), maximum(training.item))

    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    @tprogress Threads.@threads for i = 1:size(S)[1]
        S[:, i] = vec(R[:, i]' * R) ./ norms ./ norms[i]
    end

    S = convert.(Float32, S)
    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## ItemCF residuals

In [4]:
item_alphas = ["ItemCF.$K" for K in [2^4, 2^6, 2^8, 2^10, 2^12]]
alphas = ["UserItemBiases"; item_alphas]
item_cf_params = [[
        cf_params(
            name = "ItemCF.Resid.ItemCF.1.$K",
            training_residuals = alphas,
            validation_residuals = alphas,
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(alphas),
            K = K,
            λ = rand(3),
        ) for K in [2^8]
    ];
];
for param in item_cf_params
    optimize_model(param)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|███████████████████████████| Time: 0:06:05 ( 0.43  s/it)
[ Debug: 20220224 02:52:07 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.3394871342527668,0.008285988089659446,0.0008073878252998745,-0.0019224987030968866) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.6145425640558386,0.5192343366287853,-0.39556796186132487,0.4598000522906634): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff

Iter     Function value   Gradient norm 
     0     1.339487e+00     8.285988e-03
 * Current step size: 1.0
 * time: 0.029947996139526367
 * g(x): [0.008285988089659446, 0.0008073878252998745, -0.0019224987030968866]
 * x: [0.5925598025631478, 0.9936160909300036, 0.610059322281179]


Progress: 100%|███████████████████████████| Time: 0:08:02 ( 0.56  s/it)
[ Debug: 20220224 03:00:52 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.339416539274545,0.007683094923045469,0.0008419692538979136,-0.0019987145413034794) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.6113943531754378,0.5295137003544521,-0.39419893459975813,0.4538652527321935): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsD

     1     1.338922e+00     2.931328e-03
 * Current step size: 16.9497170180995
 * time: 1790.7841532230377
 * g(x): [-0.0012873527799187887, 0.0013472004322177043, -0.0029313278363520393]
 * x: [0.4521146492280773, 0.979931095767312, 0.6426451312663345]


Progress: 100%|███████████████████████████| Time: 0:06:30 ( 0.46  s/it)
[ Debug: 20220224 03:29:10 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.3387775396886037,-0.0008435514418483126,0.0010191406403909768,-0.0027004429520637863) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.5741922127281386,0.6605053257159763,-0.362481294273565,0.3551286353276957): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, Ratings

...

Progress: 100%|███████████████████████████| Time: 0:04:15 ( 0.30  s/it)
[ Debug: 20220224 06:22:59 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.3359335321443142,-4.7454866115906986e-7,-2.5688516571866443e-8,-1.2799544076113145e-9) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.9985623367321588,5.3845038789737805,-0.9347636824439814,0.10638137451647056): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, Rati

    13     1.335934e+00     4.189314e-08
 * Current step size: 0.736285287035187
 * time: 12941.118202209473
 * g(x): [-3.4633772930109583e-9, -4.189314342278179e-8, -2.915340234635521e-10]
 * x: [0.2711682271102025, 0.8715700442722762, 12.245197643632142]


Progress: 100%|███████████████████████████| Time: 0:04:12 ( 0.29  s/it)
[ Debug: 20220224 06:32:19 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.3359335321413235,-1.0865410697052847e-10,-5.374011271747621e-10,-5.563619925375903e-12) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.9985864945672036,5.384551294257945,-0.9347830383412999,0.10638695761973191): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, Rati

    14     1.335934e+00     6.099900e-11
 * Current step size: 1.0142056910796005
 * time: 13750.66489815712
 * g(x): [-6.099899698551905e-11, 5.0085579212496423e-11, -1.5012599874999186e-12]
 * x: [0.271168149942313, 0.8715720549773613, 12.245267851488217]


Progress: 100%|███████████████████████████| Time: 3:04:57 ( 9.43  s/it)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:29
[ Info: 20220224 09:52:43 training set: RMSE 1.0723335994438785 MAE 0.7735985035065189 R2 0.056260313937872564
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
[ Info: 20220224 09:53:00 validation set: RMSE 1.155825909097613 MAE 0.8352890733152193 R2 0.014468054244603623
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


## GNN residuals

In [5]:
alphas = ["UserItemBiases"; ["GNN"]] # TODO handle multiple GNN seeds
item_cf_params = [[
        cf_params(
            name = "ItemCF.Resid.GNN.1.$K",
            training_residuals = alphas,
            validation_residuals = alphas,
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(alphas),
            K = K,
            λ = rand(3),
        ) for K in [2^8]
    ];
];
for param in item_cf_params
    optimize_model(param)
end

[ Debug: 20220224 09:53:37 generating similarity matrix for ["UserItemBiases", "GNN"]
Progress: 100%|███████████████████████████| Time: 0:35:15 ( 1.79  s/it)
Progress: 100%|███████████████████████████| Time: 0:05:00 ( 0.35  s/it)
[ Debug: 20220224 10:35:57 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.383618687520694,-0.07966626834019212,-0.06719060381756264,-0.009893751578510804) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.7500523689374903,0.8719122272469155,0.7333020531364115,0.4337260982580872): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{Fo

Iter     Function value   Gradient norm 
     0     1.383619e+00     7.966627e-02
 * Current step size: 1.0
 * time: 7.891654968261719e-5
 * g(x): [-0.07966626834019212, -0.06719060381756264, -0.009893751578510804]
 * x: [0.27762955257700805, 0.07820413778423374, 0.44366666297783475]


Progress: 100%|███████████████████████████| Time: 0:05:10 ( 0.36  s/it)
[ Debug: 20220224 10:41:25 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.3722646410748596,-0.08872496384954928,-0.06723851932127929,-0.011224990836790766) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.8772711818599543,1.0000242543143179,0.6841084898774262,0.5453468789913163): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsData

     1     1.322333e+00     3.690534e-02
 * Current step size: 18.24308966946224
 * time: 1264.8195309638977
 * g(x): [0.03690533717909419, 0.02018696021413107, 0.016882266616803296]
 * x: [1.7309884295385736, 1.3039683481733408, 0.624159260191991]


Progress: 100%|███████████████████████████| Time: 0:05:06 ( 0.36  s/it)
[ Debug: 20220224 11:02:41 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.2996366130438417,0.03268724269781456,-0.00203688861394732,0.01917418562668938) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.7762804711647657,0.8009252834243794,-0.8474148420552614,1.8970817069364405): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDatase

...

Progress: 100%|███████████████████████████| Time: 0:03:25 ( 0.24  s/it)
[ Debug: 20220224 12:21:13 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.291216354384585,-0.00031957640727185265,0.00026764485750248144,-0.0001922977760588278) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.9397249845288097,0.17268182723169562,-0.4689117559194585,3.0583527706797655): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, Rati

    13     1.291215e+00     4.191925e-08
 * Current step size: 0.9526445719376411
 * time: 8311.771255970001
 * g(x): [-3.389341750451696e-8, 3.017997979493296e-8, -4.1919245698913674e-8]
 * x: [1.3104602884475052, 1.007245889221601, 0.06513771768937648]


Progress: 100%|███████████████████████████| Time: 0:03:19 ( 0.23  s/it)
[ Debug: 20220224 13:02:01 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.291215042790724,1.3271691668657584e-7,-1.1839816564662093e-7,1.637493387474294e-7) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.93569120884381,0.16871480317910248,-0.46452148436260865,3.1069333049483645): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDa

    14     1.291215e+00     1.241130e-10
 * Current step size: 1.0160635017855784
 * time: 8985.400784015656
 * g(x): [-3.567779476950461e-11, -1.300341409664028e-11, -1.241130254378301e-10]
 * x: [1.3104609221109944, 1.0072456242321344, 0.06513756327815164]


Progress: 100%|███████████████████████████| Time: 2:27:21 ( 7.51  s/it)
[ Info: 20220224 15:38:36 training set: RMSE 1.01715747223673 MAE 0.7376800701138309 R2 0.12271032105317847
[ Info: 20220224 15:38:52 validation set: RMSE 1.1363164360293956 MAE 0.8190951648851915 R2 0.10689649118414046


## MF residuals

In [5]:
alphas = ["UserItemBiases"; ["MatrixFactorization.$K" for K in [10, 20, 40]]]
item_cf_params = [[
        cf_params(
            name = "ItemCF.Resid.MF.1.$K",
            training_residuals = alphas,
            validation_residuals = alphas,
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(alphas),
            K = K,
            λ = rand(3),
        ) for K in [2^8]
    ];
];
for param in item_cf_params
    optimize_model(param)
end

[ Debug: 20220225 04:57:41 generating similarity matrix for ["UserItemBiases", "MatrixFactorization.10", "MatrixFactorization.20", "MatrixFactorization.40"]
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
Progress: 100%|███████████████████████████| Time: 0:34:30 ( 1.75  s/it)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|███████████████████████████| Time: 0:03:56 ( 0.28  s/it)
[ Debug: 20220225 05:37:17 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.3507808,-0.07799728,-0.022881081,-0.0047614267) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.2016828,1.6086196,0.07692187,0.5520

Iter     Function value   Gradient norm 
     0     1.350781e+00     7.799728e-02
 * Current step size: 1.0
 * time: 0.02466106414794922
 * g(x): Float32[-0.07799728, -0.022881081, -0.0047614267]
 * x: Float32[0.39371875, 0.6054712, 0.8064917]


Progress: 100%|███████████████████████████| Time: 0:03:46 ( 0.26  s/it)
[ Debug: 20220225 05:41:38 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.344247,-0.07471313,-0.023365023,-0.004312556) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.3348936,1.6925411,0.046648085,0.6482752): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(0.47171605,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#val

     1     1.337107e+00     2.874195e-02
 * Current step size: 15.948772
 * time: 1027.1654770374298
 * g(x): Float32[0.028741948, -0.003660889, 0.010047991]
 * x: Float32[1.6376796, 0.97039634, 0.88243055]


Progress: 100%|███████████████████████████| Time: 0:03:50 ( 0.27  s/it)
[ Debug: 20220225 05:58:37 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.326811,0.028325465,-0.0034695636,0.0101227285) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(2.4534044,1.3950583,-0.88105667,1.7996563): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.31858,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#valid

...

Progress: 100%|███████████████████████████| Time: 0:03:37 ( 0.25  s/it)
[ Debug: 20220225 08:03:13 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.3160772,-3.7210284e-5,3.8093567e-5,-6.1569546e-5) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(0.9494337,0.20405066,-0.39455682,2.9375546): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.2899092,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"

    12     1.316077e+00     3.600489e-05
 * Current step size: 1.8911698
 * time: 9233.891863107681
 * g(x): Float32[-8.553565f-6, 4.7689896f-6, -3.6004894f-5]
 * x: Float32[1.2907803, 1.0192455, 0.07642477]


Progress: 100%|███████████████████████████| Time: 0:03:31 ( 0.25  s/it)
[ Debug: 20220225 08:15:06 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.3160771,-6.007482e-7,6.239364e-7,-2.94638e-6) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(0.9492844,0.20369123,-0.39398682,2.9412904): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.2906262,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#val

    13     1.316077e+00     4.124272e-07
 * Current step size: 1.1024141
 * time: 9944.882198095322
 * g(x): Float32[2.0471322f-7, 2.0678057f-7, 4.1242717f-7]
 * x: Float32[1.2906103, 1.0191946, 0.076539166]


Progress: 100%|███████████████████████████| Time: 2:25:53 ( 7.44  s/it)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
[ Info: 20220225 10:52:33 training set: RMSE 0.9606385253560201 MAE 0.6986439551368863 R2 0.11798298505288429
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
[ Info: 20220225 10:52:44 validation set: RMSE 1.147204099141028 MAE 0.8242867631237408 R2 0.0737156569242976
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
